In [1]:
from network import NetworkModule
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import torch
import numpy as np

KeyboardInterrupt: 

In [ ]:
training_airfoils = [5,13,21,25]
validation_airfoils = [9]
testing_airfoils = [17]

In [ ]:
stopper = EarlyStopping(monitor="val_loss",
                        patience=10,
                        verbose=False,
                        mode="min",
                        min_delta=0.0001)
trainer = pl.Trainer(check_val_every_n_epoch=1,
                    callbacks=stopper,
                    max_epochs=50)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
reg_layers = [0,0,1,0,1,1]

In [5]:
net = NetworkModule(architecture='regularization',
                    train_airfoils = training_airfoils,
                    val_airfoils=validation_airfoils,
                    test_airfoils=testing_airfoils,
                    batch_size = 64,
                    learning_rate = 0.01,
                    weight_penalty=1e-3,
                    reg_layers=reg_layers)
net

NetworkModule(
  (model): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=40, out_features=20, bias=True)
      (1): ReLU()
    )
    (1-2): 2 x Sequential(
      (0): Linear(in_features=20, out_features=20, bias=True)
      (1): ReLU()
    )
    (3): Sequential(
      (0): Linear(in_features=25, out_features=20, bias=True)
      (1): ReLU()
    )
    (4): Sequential(
      (0): Linear(in_features=20, out_features=20, bias=True)
      (1): ReLU()
    )
    (5): Sequential(
      (0): Linear(in_features=25, out_features=20, bias=True)
      (1): ReLU()
    )
    (6): Sequential(
      (0): Linear(in_features=25, out_features=3, bias=True)
      (1): ReLU()
    )
  )
)

# FIRST RUN

In [5]:
for i in range(1,7):
    print(i,torch.mean(torch.abs(net.model[i][0].weight[:,-5:])))

0 tensor(0.0785, grad_fn=<MeanBackward0>)
1 tensor(0.0981, grad_fn=<MeanBackward0>)
2 tensor(0.0846, grad_fn=<MeanBackward0>)
3 tensor(0.1047, grad_fn=<MeanBackward0>)
4 tensor(0.1056, grad_fn=<MeanBackward0>)
5 tensor(0.1007, grad_fn=<MeanBackward0>)
6 tensor(0.0754, grad_fn=<MeanBackward0>)


In [36]:
for i in range(len(reg_layers)):
    if reg_layers[i]:
        l = i+1
        print(l,torch.mean(torch.abs(net.model[l][0].weight[:,-5:])))

4 tensor(0.1569, grad_fn=<MeanBackward0>)
5 tensor(0.1790, grad_fn=<MeanBackward0>)
6 tensor(0.1358, grad_fn=<MeanBackward0>)


In [23]:
# Penalty 1e-2
L1 = np.mean(torch.tensor(L1).detach().numpy(),axis=0)
for i in range(len(L1)): print(i+1,L1[i])

1 0.04863121
2 0.05347227
3 0.06300371
4 0.052765623
5 0.07467669
6 0.03457282


In [17]:
# Select 2,3,4,5
L1 = np.mean(torch.tensor(L1).detach().numpy(),axis=0)
for i in range(len(L1)): print(i+1,L1[i])

1 0.12092545
2 0.11666157
3 0.058114618
4 0.06771401
5 0.06397276
6 0.1896327


In [26]:
# Select 2,3,4,5
L1 = np.mean(torch.tensor(L1).detach().numpy(),axis=0)
for i in range(len(L1)): print(i+1,L1[i])

1 0.11631209
2 0.036322527
3 0.080626115
4 0.044734508
5 0.047170877
6 0.1252399


In [28]:
# Select 3,5
L1 = np.mean(torch.tensor(L1).detach().numpy(),axis=0)
for i in range(len(L1)): print(i+1,L1[i])

1 0.108660586
2 0.12549976
3 0.10090427
4 0.14318569
5 0.11672858
6 0.19168264


In [13]:
torch.mean(torch.abs(net.model[1][0].weight[:,:]))

tensor(0.1010, grad_fn=<MeanBackward0>)

In [25]:
net

NetworkModule(
  (model): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=40, out_features=20, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=20, out_features=20, bias=True)
      (1): ReLU()
    )
    (2-5): 4 x Sequential(
      (0): Linear(in_features=25, out_features=20, bias=True)
      (1): ReLU()
    )
    (6): Sequential(
      (0): Linear(in_features=20, out_features=3, bias=True)
      (1): ReLU()
    )
  )
)

In [35]:
trainer.test(net)

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.17981365323066711
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.17981365323066711}]

In [32]:
trainer.test(net)

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.16481004655361176
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.16481004655361176}]

In [34]:
L1 = []

for i in range(1):
    stopper = EarlyStopping(monitor="val_loss",
        patience=10)
    trainer = pl.Trainer(check_val_every_n_epoch=1,
        callbacks=stopper,
        max_epochs=50)
    
    reg_layers = [0,0,0,1,1,1]
    
    net = NetworkModule(architecture='regularization',
                    train_airfoils = training_airfoils,
                    val_airfoils=validation_airfoils,
                    test_airfoils=testing_airfoils,
                    batch_size = 64,
                    learning_rate = 0.01,
                    weight_penalty=0,
                    reg_layers=reg_layers)
    
    trainer.fit(net)
    #trainer.test(net)
    #L1.append([torch.mean(torch.abs(net.model[i+1][0].weight[:,-5:])) for i in range(6)])
    

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type       | Params
-------------------------------------
0 | model | ModuleList | 3.2 K 
-------------------------------------
3.2 K     Trainable params
0         Non-trainable params
3.2 K     Total params
0.013     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]